# Big Data Analytics of Hotel Bookings

### Data Import

In [101]:
# import modules
import pandas as pd
from pyspark.sql import SparkSession

In [102]:
# build spark session and spark context
spark = SparkSession.builder \
        .appName("hotel") \
        .getOrCreate()
sc = spark.sparkContext

In [139]:
df = spark.read.csv('hotel_bookings.csv',  inferSchema=True, header = True)
df.take(2)

[Row(hotel='Resort Hotel', is_canceled=0, lead_time=342, arrival_date_year=2015, arrival_date_month='July', arrival_date_week_number=27, arrival_date_day_of_month=1, stays_in_weekend_nights=0, stays_in_week_nights=0, adults=2, children='0', babies=0, meal='BB', country='PRT', market_segment='Direct', distribution_channel='Direct', is_repeated_guest=0, previous_cancellations=0, previous_bookings_not_canceled=0, reserved_room_type='C', assigned_room_type='C', booking_changes=3, deposit_type='No Deposit', agent='NULL', company='NULL', days_in_waiting_list=0, customer_type='Transient', adr=0.0, required_car_parking_spaces=0, total_of_special_requests=0, reservation_status='Check-Out', reservation_status_date='2015-07-01'),
 Row(hotel='Resort Hotel', is_canceled=0, lead_time=737, arrival_date_year=2015, arrival_date_month='July', arrival_date_week_number=27, arrival_date_day_of_month=1, stays_in_weekend_nights=0, stays_in_week_nights=0, adults=2, children='0', babies=0, meal='BB', country='

In [104]:
# view columns and schema
df.printSchema()

root
 |-- hotel: string (nullable = true)
 |-- is_canceled: integer (nullable = true)
 |-- lead_time: integer (nullable = true)
 |-- arrival_date_year: integer (nullable = true)
 |-- arrival_date_month: string (nullable = true)
 |-- arrival_date_week_number: integer (nullable = true)
 |-- arrival_date_day_of_month: integer (nullable = true)
 |-- stays_in_weekend_nights: integer (nullable = true)
 |-- stays_in_week_nights: integer (nullable = true)
 |-- adults: integer (nullable = true)
 |-- children: string (nullable = true)
 |-- babies: integer (nullable = true)
 |-- meal: string (nullable = true)
 |-- country: string (nullable = true)
 |-- market_segment: string (nullable = true)
 |-- distribution_channel: string (nullable = true)
 |-- is_repeated_guest: integer (nullable = true)
 |-- previous_cancellations: integer (nullable = true)
 |-- previous_bookings_not_canceled: integer (nullable = true)
 |-- reserved_room_type: string (nullable = true)
 |-- assigned_room_type: string (nullab

### Data Preprocessing

In [105]:
# replace the strings "NULL" and "NA" with null value
df_withNull = df.replace('NULL', None).replace('NA', None)

# view number of null values per column
from pyspark.sql.functions import isnan, when, count, col
df_withNull.select([count(when(col(c).isNull(), c)).alias(c) for c in df_Null.columns]).show()

+-----+-----------+---------+-----------------+------------------+------------------------+-------------------------+-----------------------+--------------------+------+--------+------+----+-------+--------------+--------------------+-----------------+----------------------+------------------------------+------------------+------------------+---------------+------------+-----+-------+--------------------+-------------+---+---------------------------+-------------------------+------------------+-----------------------+
|hotel|is_canceled|lead_time|arrival_date_year|arrival_date_month|arrival_date_week_number|arrival_date_day_of_month|stays_in_weekend_nights|stays_in_week_nights|adults|children|babies|meal|country|market_segment|distribution_channel|is_repeated_guest|previous_cancellations|previous_bookings_not_canceled|reserved_room_type|assigned_room_type|booking_changes|deposit_type|agent|company|days_in_waiting_list|customer_type|adr|required_car_parking_spaces|total_of_special_reque

In [125]:
# replace null values in 'children' to 0 since there are only 4
df2 = df_withNull.fillna({'children':0})

# replace 'children' datatype to int
df2 = df2.withColumn('children', col('children').cast("Int"))

In [126]:
# drop 'company' and 'agent' due to high null count
# drop 'hotel' because it doesn't add anything to the model and 'arrival_date_week_number' because it is redundant with dates
df2 = df2.drop('agent', 'company', 'hotel', 'arrival_date_week_number')

In [127]:
df2.dtypes

[('hotel', 'string'),
 ('is_canceled', 'int'),
 ('lead_time', 'int'),
 ('arrival_date_year', 'int'),
 ('arrival_date_month', 'string'),
 ('arrival_date_week_number', 'int'),
 ('arrival_date_day_of_month', 'int'),
 ('stays_in_weekend_nights', 'int'),
 ('stays_in_week_nights', 'int'),
 ('adults', 'int'),
 ('children', 'int'),
 ('babies', 'int'),
 ('meal', 'string'),
 ('country', 'string'),
 ('market_segment', 'string'),
 ('distribution_channel', 'string'),
 ('is_repeated_guest', 'int'),
 ('previous_cancellations', 'int'),
 ('previous_bookings_not_canceled', 'int'),
 ('reserved_room_type', 'string'),
 ('assigned_room_type', 'string'),
 ('booking_changes', 'int'),
 ('deposit_type', 'string'),
 ('days_in_waiting_list', 'int'),
 ('customer_type', 'string'),
 ('adr', 'double'),
 ('required_car_parking_spaces', 'int'),
 ('total_of_special_requests', 'int'),
 ('reservation_status', 'string'),
 ('reservation_status_date', 'string')]

**Data Preprocessing Todo's**
- map arrival_date_month from strings to numbers: so "January" = 1, etc.
- numerically encode all the other string variables, otherwise remove them  

In [144]:
# numerically encode arrival_date_month
...

In [155]:
# for 'meal': undefined/SC are same, they mean no meal package. BB is bed&breakfast, HB is breakfast and dinner, and FB is full breakfast lunch and dinner

# replace Undefined with SC in meal package
df2 = df2.replace('Undefined', 'SC')

# see value counts for 'meal'
df2.groupBy('meal').count().show()

+----+-----+
|meal|count|
+----+-----+
|  SC|11819|
|  FB|  798|
|  BB|92310|
|  HB|14463|
+----+-----+



In [146]:
# convert to RDD
# data_rdd = df2.rdd

# 70/30 test train split
# train, test = data_rdd.randomSplit([0.7, 0.3], seed=42)

### Exploratory Data Analysis

helpful links:
- https://spark.apache.org/docs/latest/ml-guide.html 
